## Overview
This query generates a billing/usage report from the `system.billing` database, focusing on Azure Databricks resources.

## Table Structure
The query joins two primary tables:
- `usage` (aliased as `u`): Contains usage data.
- `list_prices` (aliased as `p`): Contains pricing information.

## Fixed Values
| Field           | Value      |
|-----------------|------------|
| ProviderName    | Microsoft  |
| PublisherName   | Databricks |
| ServiceCategory | Analytics  |
| RegionId        | West US    |

## Cost Calculations
Cost metrics are computed using the pricing information from the `list_prices` table multiplied by the usage quantity from the `usage` table. For example:
- **ContractedCost** and **ListCost** are calculated as:  
  `p.pricing.effective_list.default * u.usage_quantity`
- **EffectiveCost** uses conditional logic based on `ChargeCategory`:  
  - For Purchase rows, the cost is `0`.  
  - For Credit rows and Usage rows, the cost is `p.pricing.effective_list.default * u.usage_quantity`.  
  - For other charge types (e.g., Tax), adjustments may apply.

When unit prices are provided, multiplying the unit price by the usage quantity should yield the corresponding cost metric, except for rows with `ChargeClass = "Correction"`, where discrepancies are permitted.

### Changed Columns in Version 1.1

https://github.com/FinOps-Open-Cost-and-Usage-Spec/FOCUS_Spec/blob/working_draft/CHANGELOG.md

Columns Added:
1. CapacityReservationId
1. CapacityReservationStatus
1. CommitmentDiscountQuantity
1. CommitmentDiscountUnit
1. ServiceSubcategory
1. SkuMeter
1. SkuPriceDetails

In [ ]:
%sql
USE system.billing;

SELECT
    '' AS AvailabilityZone,                    -- dimension, recommended, nulls, string
    u.account_id AS BillingAccountId,          -- dimension, mandatory, not null, string (This is the Databrick's Account ID)
    '' AS BillingAccountName,                  -- dimension, mandatory, nulls, string
    p.currency_code AS BillingCurrency,        -- dimension, mandatory, not null, currency (Currency Code)
    date_format(date_trunc('month', u.usage_date), 'yyyy-MM-dd\'T\'HH:mm:ss\'Z\'') AS BillingPeriodStart,   -- dimension, mandatory, not null, datetime (ISO 8601)
    date_format(date_add(last_day(u.usage_date), 1), 'yyyy-MM-dd\'T\'HH:mm:ss\'Z\'') AS BillingPeriodEnd,   -- dimension, mandatory, not null, datetime (ISO 8601)
    p.pricing.effective_list.default * u.usage_quantity AS BilledCost,      -- metric, mandatory, not null, numeric
    NULL AS CapacityReservationId,             -- dimension, optional, nulls, string
    NULL AS CapacityReservationStatus,         -- dimension, optional, nulls, string [Used, Unused]
    'Usage' AS ChargeCategory,                 -- dimension, mandatory, not null, string [Usage, Purchase, Tax, Credit, Adjustment]
    NULL AS ChargeClass,                       -- dimension, mandatory, nulls, string [Correction]
    u.usage_type AS ChargeDescription,         -- dimension, mandatory, nulls, string [Databricks Docs: COMPUTE_TIME, STORAGE_SPACE, NETWORK_BYTES, NETWORK_HOUR, API_OPERATION, TOKEN, GPU_TIME] 
    CASE
        WHEN ChargeCategory = 'Purchase' THEN 'One-Time'
        ELSE 'Usage-Based'
    END AS ChargeFrequency,                    -- dimension, recommended, not null, string [One-Time, Recurring, Usage-Based]
    date_format(u.usage_end_time, 'yyyy-MM-dd\'T\'HH:mm:ss\'Z\'') AS ChargePeriodEnd,      -- dimension, mandatory, not null, datetime (ISO 8601)
    date_format(u.usage_start_time, 'yyyy-MM-dd\'T\'HH:mm:ss\'Z\'') AS ChargePeriodStart,  -- dimension, mandatory, not null, datetime (ISO 8601)
    NULL AS CommitmentDiscountCategory,        -- dimension, conditional, nulls, string [Spend, Usage]
    NULL AS CommitmentDiscountID,              -- dimension, conditional, nulls, string
    NULL AS CommitmentDiscountName,            -- dimension, conditional, nulls, string
    NULL AS CommitmentDiscountQuantity,        -- metric, optional, nulls, decimal
    NULL AS CommitmentDiscountStatus,          -- dimension, conditional, nulls, string [Used, Unsued]
    NULL AS CommitmentDiscountUnit,            -- dimension, optional, nulls, string
    CASE
        WHEN ChargeCategory = 'Usage'
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
        THEN u.usage_quantity
        ELSE NULL
    END AS ConsumedQuantity,                   -- metric, conditional, nulls, decimal
    CASE
        WHEN ChargeCategory = 'Usage'
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
        THEN u.usage_unit
        ELSE NULL
    END AS ConsumedUnit,                       -- metric, conditional, nulls, string
    p.pricing.effective_list.default * u.usage_quantity AS ContractedCost,  -- metric, mandatory, not null, decimal (The column needs thorough review)
    NULL AS ContractedUnitPrice,                -- metric, conditional, nulls, decimal
    CASE 
        WHEN ChargeCategory = 'Purchase' THEN 0
        WHEN ChargeCategory = 'Usage' THEN p.pricing.effective_list.default * u.usage_quantity
        WHEN ChargeCategory = 'Credit' THEN p.pricing.effective_list.default * u.usage_quantity
        ELSE p.pricing.effective_list.default * u.usage_quantity  -- For other charge types (e.g. Tax), adjust aggregation as needed
    END AS EffectiveCost,                      -- metric, mandatory, not null, numeric
    u.cloud AS InvoiceIssuerName,              -- dimension, mandatory, not null, string
    p.pricing.effective_list.default * u.usage_quantity AS ListCost,        -- metric, mandatory, not null, decimal
    CASE
        WHEN ChargeCategory = 'Tax' THEN NULL
        WHEN ChargeCategory IN ('Usage', 'Purchase')
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
            THEN CAST(p.pricing.effective_list.default AS DECIMAL(38,10))
        ELSE NULL
    END AS ListUnitPrice,                      -- metric, conditional, nulls, decimal
    CASE
        -- When pricing is based on tax, PricingCategory MUST be null.
        WHEN ChargeCategory = 'Tax' THEN NULL

        -- For Usage or Purchase (with non-Correction ChargeClass), PricingCategory MUST not be null.
        WHEN ChargeCategory IN ('Usage','Purchase')
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
            THEN 
                CASE
                    -- When a CommitmentDiscountID is present, use 'Committed'
                    WHEN CommitmentDiscountID IS NOT NULL THEN 'Committed'
                    -- (Assuming predetermined pricing in absence of another indicator)
                    ELSE 'Standard'
                END

        -- For all other combinations:
        ELSE 
            CASE
                -- If CommitmentDiscountID is present outside Usage/Purchase, still return 'Committed'
                WHEN CommitmentDiscountID IS NOT NULL THEN 'Committed'
                -- Otherwise, if a pricing model exists but no allowed value applies, return 'Other'
                ELSE 'Other'
            END
    END AS PricingCategory,                    -- dimension, conditional, nulls, string [Standard, Dynamic, Committed, Other]
    CASE
        -- Must be null for Tax rows.
        WHEN ChargeCategory = 'Tax' THEN NULL

        -- For Usage or Purchase with non-Correction ChargeClass, value must not be null.
        WHEN ChargeCategory IN ('Usage', 'Purchase')
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction') THEN u.usage_quantity

        -- For Correction rows, we allow a value (which may be negative) but discrepancies between cost calculations are permitted.
        WHEN ChargeClass = 'Correction' THEN u.usage_quantity

        -- For all other combinations, the value may be null.
        ELSE NULL
    END AS PricingQuantity,                    -- metric, mandatory, nulls, decimal
    CASE
        WHEN ChargeCategory = 'Tax' THEN NULL
        WHEN ChargeCategory IN ('Usage', 'Purchase')
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
            THEN u.usage_unit
        ELSE NULL
    END AS PricingUnit,                        -- dimension, mandatory, nulls, string
    'Microsoft' AS ProviderName,               -- dimension, mandatory, not null, string
    'Databricks' AS PublisherName,             -- dimension, mandatory, not null, string
    'West US' AS RegionId,                     -- dimension, conditional, nulls, string
    '' AS RegionName,                          -- dimension, conditional, nulls, string
    '' AS ResourceId,                          -- dimension, conditional, nulls, string
    '' AS ResourceName,                        -- dimension, conditional, nulls, string
    CASE 
        WHEN u.billing_origin_product IS NOT NULL THEN u.billing_origin_product
        ELSE NULL
    END AS ResourceType,                       -- dimension, conditional, nulls, string
    'Analytics' AS ServiceCategory,            -- dimension, mandatory, not null, string [AI and Machine Learning, Analytics, Business Applications, Compute, Databases, Developer Tools, Multicloud, Identity, Integration, Internet of Things, Management and Governance, Media, Migration, Mobile, Networking, Security, Storage, Web, Other]
    u.billing_origin_product || ' | ' || u.sku_name AS ServiceName,  -- dimension, mandatory, not null, string
    'Analytics Platforms' AS ServiceSubcategory,                     -- dimension, optional, not null, string [AI Platforms, Bots, Generative AI, Machine Learning, Natural Language Processing, Other (AI and Machine Learning), Analytics Platforms, Business Intelligence, Data Processing, Search, Streaming Analytics, Other (Analytics), Productivity and Collaboration, Other (Business Applications), Containers, End User Computing, Quantum Compute, Serverless Compute, Virtual Machines, Other (Compute), Caching, Data Warehouses, Ledger Databases, NoSQL Databases, Relational Databases, Time Series Databases, Other (Databases), Developer Platforms, Continuous Integration and Deployment, Development Environments, Source Code Management, Quality Assurance, Other (Developer Tools), Identity and Access Management, Other (Identity), API Management, Messaging, Workflow Orchestration, Other (Integration), IoT Analytics, IoT Platforms, Other (Internet of Things), Architecture, Compliance, Cost Management, Data Governance, Disaster Recovery, Endpoint Management, Observability, Support, Other (Management and Governance), Content Creation, Gaming, Media Streaming, Mixed Reality, Other (Media), Data Migration, Resource Migration, Other (Migration), Other (Mobile), Multicloud Integration, Other (Multicloud), Application Networking, Content Delivery, Network Connectivity, Network Infrastructure, Network Routing, Network Security, Other (Networking), Secret Management, Security Posture Management, Threat Detection and Response, Other (Security), Backup Storage, Block Storage, File Storage, Object Storage, Storage Platforms, Other (Storage), Application Platforms, Other (Web), Other (Other)]
    CASE
        -- When ChargeCategory is Tax, SkuId must be null.
        WHEN ChargeCategory = 'Tax' THEN NULL

        -- When ChargeCategory is Usage or Purchase and ChargeClass is not "Correction",
        -- SkuId must be non-null (populated from u.sku_name).
        WHEN ChargeCategory IN ('Usage','Purchase') 
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
            THEN u.sku_name

        -- For all other combinations, SkuId is allowed to be null;
        -- also, when an overarching SKU ID is not supported, SkuId equals SKUPriceId (here assumed to be u.sku_name).
        ELSE u.sku_name
    END AS SkuId,                              -- dimension, conditional, nulls, string                       
    CASE
        -- When ChargeCategory is Tax, SKUPriceID must be null.
        WHEN ChargeCategory = 'Tax' THEN NULL

        -- When ChargeCategory is Usage or Purchase and ChargeClass is not "Correction",
        -- SKUPriceID must be non-null (populated from u.sku_name).
        WHEN ChargeCategory IN ('Usage','Purchase')
            AND (ChargeClass IS NULL OR ChargeClass <> 'Correction')
            THEN u.sku_name

        -- For all other combinations, SKUPriceID equals u.sku_name.
        ELSE u.sku_name
    END AS SKUPriceID,                         -- dimension, conditional, nulls, string
    NULL AS SkuMeter,                          -- dimension, optional, nulls, string
    NULL AS SkuPriceDetails,                   -- dimension, optional, nulls, json (key value)
    CASE 
        WHEN u.workspace_id IS NOT NULL 
            AND TRIM(u.workspace_id) <> '' 
            THEN u.workspace_id 
        ELSE NULL 
    END AS SubAccountId,                       -- dimension, conditional, nulls, string
    CASE 
    WHEN u.workspace_id IS NOT NULL 
            AND TRIM(u.workspace_id) <> '' 
            THEN u.workspace_id 
        ELSE NULL 
    END AS SubAccountName,                     -- dimension, mandatory, not null, string
    COALESCE(u.custom_tags, CAST(map() AS MAP<STRING, STRING>)) AS Tags, -- dimension, conditional, nulls, json (key value)
    CASE 
        WHEN CommitmentDiscountID IS NULL THEN NULL
        ELSE 'Commitment'
    END AS CommitmentDiscountType,             -- dimension, conditional, nulls, string
    u.usage_date AS UnmappedUsageDate          -- ?? should interogate these data
FROM
    usage u
    JOIN list_prices p ON u.sku_name = p.sku_name
WHERE
    p.price_start_time <= CURRENT_DATE
    AND p.price_end_time IS NULL
    AND usage_start_time BETWEEN :start_date AND :end_date;